<a href="https://colab.research.google.com/github/hejazn86/Stock_price_prediction/blob/main/stock_data_extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Extracting Stock data from Yahoo API**

In [2]:
#!pip install yfinance
#import yfinance as yf

# Define the stock ticker and date range
ticker = 'AAPL'
start_date = '2014-01-01'
end_date = '2024-06-20'

# Fetch historical stock data of specific company (Apple)
stock_data = yf.download(ticker, start=start_date, end=end_date)

# Save the data
stock_data.to_csv('stock_price.csv')
len(stock_data)

2633

**Extracting Stock data using Alpha vantage API**

In [49]:
# Install and load Alpha Vantage API
!pip install alpha_vantage
from alpha_vantage.timeseries import TimeSeries
import pandas as pd

# Your API key
api_key = 'WL313PFE45NS5WRT'

# Initialize the TimeSeries class
ts = TimeSeries(key=api_key, output_format='pandas')

# Get historical data for a given ticker
ticker = 'AAPL'
data, meta_data = ts.get_daily(symbol=ticker, outputsize='full')

data.to_csv('stock_vantage.csv')
#print(data.head())
len(data)

6201